# 芯片资源调配

## 单个芯片

资源调度方法出自<span style="color:red">神经元计算机操作系统的资源分配方法</span>，语言选用LAVA

In [2]:
class Event:
    def __init__(self, spike=None, count=0, next_event=None):
        self.spike = spike
        self.count = count
        self.next = next_event

In [5]:
class Subgraph:
    def __init__(self, cfg_info=None):
        self.cfg_info = cfg_info
        self.ar_event_head = None
        self.ar_event_tail = None
        self.le_event_head = None
        self.le_event_tail = None

    def add_ar_event(self, event):
        if not self.ar_event_head:
            self.ar_event_head = self.ar_event_tail = event
        else:
            self.ar_event_tail.next = event
            self.ar_event_tail = event

    def add_le_event(self, event):
        if not self.le_event_head:
            self.le_event_head = self.le_event_tail = event
        else:
            self.le_event_tail.next = event
            self.le_event_tail = event


In [6]:
class Graph:
    def __init__(self):
        self.subgraphs = []  # 存储所有 subgraph
        self.edges = {}  # 键是 subgraph, 值是一个列表，包含可以达到的其他 subgraph

    def add_subgraph(self, subgraph):
        self.subgraphs.append(subgraph)
        self.edges[subgraph] = []

    def add_edge(self, from_subgraph, to_subgraph):
        self.edges[from_subgraph].append(to_subgraph)

In [7]:
from collections import deque

class BFS:
    def __init__(self, graph, start_subgraph):
        self.graph = graph
        self.start_subgraph = start_subgraph

    def search(self):
        visited = set()
        queue = deque([self.start_subgraph])

        while queue:
            current = queue.popleft()
            if current not in visited:
                print(f"Visiting Subgraph: {current.cfg_info}")
                event = current.ar_event_head #遍历event
                while event:
                    print(f"Event: {event.spike} {event.count}")
                    event = event.next
                visited.add(current)
                for neighbor in self.graph.edges[current]:
                    if neighbor not in visited:
                        queue.append(neighbor)

In [8]:
# 创建Subgraph实例
subgraph1 = Subgraph(cfg_info="Subgraph 1")
subgraph2 = Subgraph(cfg_info="Subgraph 2")
subgraph3 = Subgraph(cfg_info="Subgraph 3")
subgraph4 = Subgraph(cfg_info="Subgraph 4")

# 创建Event实例
event1 = Event(spike="Event 1 Spike", count=1)
event2 = Event(spike="Event 2 Spike", count=2)
event3 = Event(spike="Event 3 Spike", count=3)
event4 = Event(spike="Event 4 Spike", count=4)

# 将事件添加到Subgraph中
subgraph1.add_ar_event(event1)
subgraph2.add_ar_event(event2)
subgraph3.add_ar_event(event3)
subgraph4.add_ar_event(event4)

# 创建Graph实例
graph = Graph()

# 添加Subgraph到Graph
graph.add_subgraph(subgraph1)
graph.add_subgraph(subgraph2)
graph.add_subgraph(subgraph3)
graph.add_subgraph(subgraph4)

# 设置边，定义图的结构
graph.add_edge(subgraph1, subgraph2)
graph.add_edge(subgraph1, subgraph3)
graph.add_edge(subgraph2, subgraph4)
graph.add_edge(subgraph3, subgraph4)


In [9]:
# 创建BFS实例，从subgraph1开始搜索
bfs = BFS(graph, subgraph1)

# 执行搜索
bfs.search()


Visiting Subgraph: Subgraph 1
Event: Event 1 Spike 1
Visiting Subgraph: Subgraph 2
Event: Event 2 Spike 2
Visiting Subgraph: Subgraph 3
Event: Event 3 Spike 3
Visiting Subgraph: Subgraph 4
Event: Event 4 Spike 4


## 多核资源调度
资源随处理器核，任务随处理器核


### 分布式调度
每个核各有一个子图队列，子图扩展按核进行
多任务资源调度，按进程调度策略进行

### 集中调度
多个核共用一个子图队列
进程调度根据共享子图队列状态进行